## Heart Disease Classification

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, f1_score
import xgboost as xgb
from xgboost import XGBClassifier

**Data Source:** [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/45/heart+disease)

**Columns in data:**
- `age`: age in years
- `sex`: sex (1 = male; 0 = female)
- `cp`: chest pain type
    - Value 1: typical angina
    - Value 2: atypical angina
    - Value 3: non-anginal pain
    - Value 4: asymptomatic
- `trestbps`: resting blood pressure (in mm Hg on admission to the hospital)
- `chol`: serum cholesterol in mg/dl
- `fbs`: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
- `restecg`: resting electrocardiographic results
    - Value 0: normal
    - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
- `thalach`: maximum heart rate achieved
- `exang`: exercise induced angina (1 = yes; 0 = no)
- `oldpeak`: ST depression induced by exercise relative to rest
- `slope`: the slope of the peak exercise ST segment
    - Value 1: upsloping
    - Value 2: flat
    - Value 3: downsloping
- `ca`: number of major vessels (0-3) colored by flourosopy
- `thal`: 3 = normal; 6 = fixed defect; 7 = reversable defect
- `num`: diagnosis of heart disease (angiographic disease status)
    - Value 0: < 50% diameter narrowing
    - Value 1, 2, 3, or 4: > 50% diameter narrowing

## Loading Data

In [2]:
hd_hungary = pd.read_excel("hd_hungary.xlsx", header=None)
hd_switzerland = pd.read_excel("hd_switzerland.xlsx", header=None)

hd = pd.concat([hd_hungary, hd_switzerland], ignore_index=True)

In [3]:
colnames = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"]

hd.columns = colnames
hd.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,40,1,2,140,289,0,0,172,0,0.0,-9,-9,-9,0
1,49,0,3,160,180,0,0,156,0,1.0,2,-9,-9,1
2,37,1,2,130,283,0,1,98,0,0.0,-9,-9,-9,0
3,48,0,4,138,214,0,0,108,1,1.5,2,-9,-9,3
4,54,1,3,150,-9,0,0,122,0,0.0,-9,-9,-9,0


In the heart disease Switzerland dataset, missing values are labeled with '?' so I am going to change these to nulls instead.

In [4]:
hd.replace('?', np.nan, inplace=True)

In [5]:
hd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       417 non-null    int64  
 1   sex       417 non-null    int64  
 2   cp        417 non-null    int64  
 3   trestbps  415 non-null    float64
 4   chol      417 non-null    int64  
 5   fbs       342 non-null    float64
 6   restecg   416 non-null    float64
 7   thalach   416 non-null    float64
 8   exang     416 non-null    float64
 9   oldpeak   411 non-null    float64
 10  slope     400 non-null    float64
 11  ca        299 non-null    float64
 12  thal      365 non-null    float64
 13  num       417 non-null    int64  
dtypes: float64(9), int64(5)
memory usage: 45.7 KB


I am going to drop the columns for `slope`, `ca`, and `thal` because there are too many rows with missing values (labeled as "-9" in `slope` and `thal`).

In [6]:
print('Unique Values: slope')
value_counts = hd['slope'].value_counts()

print(value_counts)

print('\nUnique Values: thal')
value_counts = hd['thal'].value_counts()

print(value_counts)

print('\nUnique Values: chol')
value_counts = hd['chol'].value_counts()

print(value_counts)

Unique Values: slope
-9.0    190
 2.0    152
 1.0     45
 3.0     13
Name: slope, dtype: int64

Unique Values: thal
-9.0    266
 7.0     53
 3.0     26
 6.0     20
Name: thal, dtype: int64

Unique Values: chol
 0      123
-9       23
 246      5
 230      5
 275      5
       ... 
 85       1
 329      1
 210      1
 307      1
 466      1
Name: chol, Length: 155, dtype: int64


Will also convert -9 to null values here as an extra measure

In [7]:
hd.replace(-9, np.nan, inplace=True)

In [8]:
# dropping columns
columns_to_drop = ['slope', 'ca', 'fbs', 'thal']

hd = hd.drop(columns=columns_to_drop, axis=1)

### Converting Columns to Categorical

There are many columns here that use integer codes to label certain things. I am going to swap out these integers with the labels that they stand for. Technically, I could have just converted the integer columns into strings with numbers, but I think replacing them with the labels themselves can help with explanability.

In [9]:
def num_to_cat(column, mapping):
    hd[column].replace(mapping, inplace=True)

In [10]:
cat_columns = ['sex', 'cp', 'restecg', 'exang']

cat_values = [{1: 'M', 0: 'F'},
             {1: 'typical angina', 2: 'atypical angina', 3: 'non-anginal pain', 4: 'asymptomatic'},
             {0: 'normal', 1: 'ST-T abnorm', 2: 'left ventricular hypertrophy'},
             {1: 'Yes', 0: 'No'}]

In [11]:
for i in range(0,len(cat_columns)):
    num_to_cat(cat_columns[i], cat_values[i])

In [12]:
hd.head()

,age,sex,cp,trestbps,chol,restecg,thalach,exang,oldpeak,num
0,40,M,atypical angina,140.0,289.0,normal,172.0,No,0.0,0
1,49,F,non-anginal pain,160.0,180.0,normal,156.0,No,1.0,1
2,37,M,atypical angina,130.0,283.0,ST-T abnorm,98.0,No,0.0,0
3,48,F,asymptomatic,138.0,214.0,normal,108.0,Yes,1.5,3
4,54,M,non-anginal pain,150.0,NaN,normal,122.0,No,0.0,0


I also want to replace all non-zero values with 1, to indicate heart disease.

In [13]:
hd.loc[hd['num'] != 0, 'num'] = 1

### Drop null rows

After looking at how many null values are left, I think the amount is small enough to drop the remaining rows that contain null values.

In [14]:
null_counts = hd.isnull().sum()

print(null_counts)

age          0
sex          0
cp           0
trestbps     3
chol        23
restecg      2
thalach      2
exang        2
oldpeak      6
num          0
dtype: int64


In [15]:
hd.dropna(inplace=True)

In [16]:
X = hd.drop('num', axis=1)

### Taking one more look

In [17]:
hd.head(10)

,age,sex,cp,trestbps,chol,restecg,thalach,exang,oldpeak,num
0,40,M,atypical angina,140.0,289.0,normal,172.0,No,0.0,0
1,49,F,non-anginal pain,160.0,180.0,normal,156.0,No,1.0,1
2,37,M,atypical angina,130.0,283.0,ST-T abnorm,98.0,No,0.0,0
3,48,F,asymptomatic,138.0,214.0,normal,108.0,Yes,1.5,1
5,39,M,non-anginal pain,120.0,339.0,normal,170.0,No,0.0,0
6,45,F,atypical angina,130.0,237.0,normal,170.0,No,0.0,0
7,54,M,atypical angina,110.0,208.0,normal,142.0,No,0.0,0
8,37,M,asymptomatic,140.0,207.0,normal,130.0,Yes,1.5,1
9,48,F,atypical angina,120.0,284.0,normal,120.0,No,0.0,0
10,37,F,non-anginal pain,130.0,211.0,normal,142.0,No,0.0,0


### Data Balance/Imbalance?

This dataset seems to be more or less balanced between 0 and 1, so I am not going to do any rebalancing.

In [18]:
y = hd['num']
y.value_counts()

1    208
0    177
Name: num, dtype: int64

## Modeling

### Train Validate Test Split

First I will one-hot encode categorical variables

In [19]:
cat_columns = ['sex', 'cp', 'restecg', 'exang']

X = pd.get_dummies(X, columns = cat_columns, drop_first=True)

Then I can split the data (20% test, 20% validate)

In [20]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

X_train, X_validate, y_train, y_validate = train_test_split(X_temp, y_temp, test_size=0.25, random_state=2023)

Scaling data

In [21]:
scaler = StandardScaler()

X_train_processed = scaler.fit_transform(X_train)

X_validate_processed = scaler.transform(X_validate)

X_test_processed = scaler.transform(X_test)

In [22]:
model_performance = pd.DataFrame(columns = ['Model', 'Accuracy', 'AUC', 'F1'])

### Logistic Regression Model

In [23]:
logistic_regression_model = LogisticRegression()

logistic_regression_model.fit(X_train_processed, y_train)

LogisticRegression()

In [24]:
y_validate_pred = logistic_regression_model.predict(X_validate_processed)

lr_accuracy_score = accuracy_score(y_validate, y_validate_pred)
lr_classification = classification_report(y_validate, y_validate_pred)
lr_auc = roc_auc_score(y_validate, y_validate_pred)
lr_f1 = f1_score(y_validate, y_validate_pred)

model_performance.loc[len(model_performance)] = ['Logistic Regression', lr_accuracy_score, lr_auc, lr_f1]

#### Model Performance

In [25]:
print("Validation Set Accuracy:", round(lr_accuracy_score, 3))
print("\nValidation Set Classification Report:\n", lr_classification)
print("Validation AUC:", round(lr_auc, 3))
print("\nValidation Set F1 Score:", round(lr_f1, 3))

Validation Set Accuracy: 0.792

Validation Set Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.82      0.78        34
           1       0.85      0.77      0.80        43

    accuracy                           0.79        77
   macro avg       0.79      0.80      0.79        77
weighted avg       0.80      0.79      0.79        77

Validation AUC: 0.795

Validation Set F1 Score: 0.805


### Random Forest

In [26]:
params = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False],
    'max_features': ['auto', 'sqrt']
}

In [27]:
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=0),
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=5,
                                   random_state=0,
                                  )

In [28]:
random_search.fit(X_train_processed, y_train)

print("Best Parameters:", random_search.best_params_)
best_rf = random_search.best_estimator_

Best Parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}


In [29]:
y_validate_pred = best_rf.predict(X_validate_processed)

rf_accuracy_score = accuracy_score(y_validate, y_validate_pred)
rf_classification = classification_report(y_validate, y_validate_pred)
rf_auc = roc_auc_score(y_validate, y_validate_pred)
rf_f1 = f1_score(y_validate, y_validate_pred)

model_performance.loc[len(model_performance)] = ['Random Forest', rf_accuracy_score, rf_auc, rf_f1]

#### Model Performance

In [30]:
print("Validation Set Accuracy:", round(rf_accuracy_score, 3))
print("\nValidation Set Classification Report:\n", rf_classification)
print("Validation AUC:", round(rf_auc, 3))
print("\nValidation Set F1 Score:", round(rf_f1, 3))

Validation Set Accuracy: 0.844

Validation Set Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.79      0.82        34
           1       0.84      0.88      0.86        43

    accuracy                           0.84        77
   macro avg       0.84      0.84      0.84        77
weighted avg       0.84      0.84      0.84        77

Validation AUC: 0.839

Validation Set F1 Score: 0.864


### XGBoost

In [31]:
params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'gamma': [0, 0.1],
    'colsample_bytree': [0.6, 0.7],
    'subsample': [0.6, 0.7]
}

In [32]:
random_search = RandomizedSearchCV(xgb.XGBClassifier(objective='binary:logistic', random_state=0),
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=5,
                                   random_state=0)

In [33]:
random_search.fit(X_train_processed, y_train)
best_xgb = random_search.best_estimator_

In [34]:
y_validate_pred = best_xgb.predict(X_validate_processed)

xgb_accuracy_score = accuracy_score(y_validate, y_validate_pred)
xgb_classification = classification_report(y_validate, y_validate_pred)
xgb_auc = roc_auc_score(y_validate, y_validate_pred)
xgb_f1 = f1_score(y_validate, y_validate_pred)

model_performance.loc[len(model_performance)] = ['XGBoost', xgb_accuracy_score, xgb_auc, xgb_f1]

#### Model Performance

In [35]:
print("Validation Set Accuracy:", round(xgb_accuracy_score, 3))
print("\nValidation Set Classification Report:\n", xgb_classification)
print("Validation AUC:\n", round(xgb_auc, 3))
print("\nValidation Set F1 Score:", round(xgb_f1, 3))

Validation Set Accuracy: 0.818

Validation Set Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.85      0.81        34
           1       0.87      0.79      0.83        43

    accuracy                           0.82        77
   macro avg       0.82      0.82      0.82        77
weighted avg       0.82      0.82      0.82        77

Validation AUC:
 0.822

Validation Set F1 Score: 0.829


## Compare Models

In [36]:
model_performance

,Model,Accuracy,AUC,F1
0,Logistic Regression,0.792208,0.795486,0.804878
1,Random Forest,0.844156,0.838919,0.863636
2,XGBoost,0.818182,0.821819,0.829268


The best performing model appears to be the **Random Forest**. Here are its performance metrics on the test set.

In [37]:
y_pred = best_rf.predict(X_test_processed)

rf_accuracy_score = accuracy_score(y_test, y_pred)
rf_classification = classification_report(y_test, y_pred)
rf_auc = roc_auc_score(y_test, y_pred)
rf_f1 = f1_score(y_test, y_pred)

In [38]:
print("Validation Set Accuracy:", round(rf_accuracy_score, 3))
print("\nValidation Set Classification Report:\n", rf_classification)
print("Validation AUC:", round(rf_auc, 3))
print("\nValidation Set F1 Score:", round(rf_f1, 3))

Validation Set Accuracy: 0.766

Validation Set Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.55      0.67        33
           1       0.73      0.93      0.82        44

    accuracy                           0.77        77
   macro avg       0.79      0.74      0.74        77
weighted avg       0.79      0.77      0.75        77

Validation AUC: 0.739

Validation Set F1 Score: 0.82


The Accuracy and AUC are quite a bit worse in the test set

## Investigating Overfitting, maybe RandomSearch is too much?

Because the data size is relatively small, I think that hyperparameter tuning these models is overfitting on the validate data. I am going to rerun these models without using random search.

I never did random search on logistic regression originally, so I am going to remove the random search from random forest and xgboost.

In [39]:
model_performance_2 = pd.DataFrame(columns = ['Model', 'Accuracy', 'AUC', 'F1'])

### Logistic Regression

In [40]:
logistic_regression_model = LogisticRegression()

logistic_regression_model.fit(X_train_processed, y_train)

y_validate_pred = logistic_regression_model.predict(X_validate_processed)

In [41]:
lr_accuracy_score = accuracy_score(y_validate, y_validate_pred)
lr_classification = classification_report(y_validate, y_validate_pred)
lr_auc = roc_auc_score(y_validate, y_validate_pred)
lr_f1 = f1_score(y_validate, y_validate_pred)

model_performance_2.loc[len(model_performance_2)] = ['Logistic Regression', lr_accuracy_score, lr_auc, lr_f1]

### Random Forest

In [42]:
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=14)
random_forest_model.fit(X_train_processed, y_train)

y_validate_pred = random_forest_model.predict(X_validate_processed)

In [43]:
rf_accuracy_score = accuracy_score(y_validate, y_validate_pred)
rf_classification = classification_report(y_validate, y_validate_pred)
rf_auc = roc_auc_score(y_validate, y_validate_pred)
rf_f1 = f1_score(y_validate, y_validate_pred)

model_performance_2.loc[len(model_performance_2)] = ['Random Forest', rf_accuracy_score, rf_auc, rf_f1]

### XGboost

In [44]:
xgboost_model = XGBClassifier(n_estimators=100, random_state=14)
xgboost_model.fit(X_train_processed, y_train)

y_validate_pred = xgboost_model.predict(X_validate_processed)

In [45]:
xgb_accuracy_score = accuracy_score(y_validate, y_validate_pred)
xgb_classification = classification_report(y_validate, y_validate_pred)
xgb_auc = roc_auc_score(y_validate, y_validate_pred)
xgb_f1 = f1_score(y_validate, y_validate_pred)

model_performance_2.loc[len(model_performance_2)] = ['XGBoost', xgb_accuracy_score, xgb_auc, xgb_f1]

### Comparing Models

In [46]:
model_performance_2

,Model,Accuracy,AUC,F1
0,Logistic Regression,0.792208,0.795486,0.804878
1,Random Forest,0.844156,0.841997,0.860465
2,XGBoost,0.753247,0.751368,0.776471


Random forest still performs the best, let's compare to the test set.

### Random Forest

In [47]:
y_pred = random_forest_model.predict(X_test_processed)

rf_accuracy_score = accuracy_score(y_test, y_pred)
rf_classification = classification_report(y_test, y_pred)
rf_auc = roc_auc_score(y_test, y_pred)
rf_f1 = f1_score(y_test, y_pred)

print("Test Set Accuracy:", round(rf_accuracy_score, 3))
print("\nTest Set Classification Report:\n", rf_classification)
print("Test AUC:", round(rf_auc, 3))
print("\nTest Set F1 Score:", round(rf_f1, 3))

Test Set Accuracy: 0.779

Test Set Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.58      0.69        33
           1       0.75      0.93      0.83        44

    accuracy                           0.78        77
   macro avg       0.80      0.75      0.76        77
weighted avg       0.80      0.78      0.77        77

Test AUC: 0.754

Test Set F1 Score: 0.828


AUC and accuracy still drop here. I think that the small sample size is leading to overfitting on my train data.